In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [10]:
weekend_Jul_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]

In [11]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in weekend_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jul_list)

25920

In [13]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 1080)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
x[:,:2]

matrix([[ 1258.89024531,  1201.74712093],
        [ 1194.93780542,  1238.17293138],
        [ 1502.40689022,  1494.93635626],
        [ 1603.53657513,  1580.34259714],
        [ 1577.71479814,  1573.80777583],
        [ 1454.47529121,  1536.92167653],
        [ 1282.88403361,  1304.0562683 ],
        [ 1316.01687384,  1270.31106743],
        [ 1714.87378275,  1670.24551043],
        [ 1658.36933114,  1710.70823719],
        [ 1379.07932164,  1464.64591842],
        [ 1219.23085193,  1252.6758417 ],
        [ 1220.91763612,  1185.42650029],
        [ 1403.42682395,  1340.54667541],
        [ 1221.31801238,  1231.51334839],
        [  994.11196795,   934.13661458],
        [ 1287.37499386,  1265.70771788],
        [ 1263.88313256,  1276.02890664],
        [ 1524.82573109,  1534.85972907],
        [ 1466.7556654 ,  1552.45551973],
        [ 1454.61141155,  1456.04710115],
        [ 1354.78903309,  1350.41971678],
        [ 1275.23711794,  1277.92862194],
        [ 1295.40842208,  1254.123

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 1080)

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 2e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 25
 AA' NZ     : 3.000e+02
 Factor NZ  : 3.250e+02
 Factor Ops : 5.525e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.20675904e+07  0.00000000e+00  7.85e+03 1.25e+03  1.00e+06     0s
   1  -6.21270921e+06 -7.07104069e+06  9.76e+02 1.56e+02  2.03e+05     0s
   2  -4.43580276e+06 -5.67739594e+06  3.62e+01 5.77e+00  2.74e+04     0s
   3  -4.58802554e+06 -4.69325391e+06  3.62e-05 5.77e-06  1.8

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[2.9108955310611194e-12,
 1432.0005386537175,
 3.228419669820923e-12,
 0.00018705455072470095,
 1492.407514252264,
 0.0003511228329137056,
 9.742406677657194e-10,
 709.0169640595683,
 1.5798519180442592e-11,
 1337.4025180235944,
 1.5798520209690607e-11,
 1.579851569011057e-11,
 1.579851846503724e-11,
 1.5889118074744406e-11,
 1697.0272345435044,
 0.00032416418691980087,
 0.0023491447420103673,
 1579.8886582804473,
 0.00012740141250944248,
 0.00034138973318409427,
 0.00023977772430856803,
 4.819283553175171e-07,
 1293.1389827285673,
 0.00022587776115499947,
 0.003401022544535781,
 1228.1939924493013,
 0.0038554224376168185,
 1212.2529460460225,
 292.8831001094382,
 0.00016936201287617164,
 0.00022940532027532212,
 1167.3545547690867,
 102.91128073215057,
 46.66408983053495,
 1312.3473667459382,
 3.7850353492304247e-07,
 3.780541677966202e-07,
 3.785099189049679e-07,
 0.0023779860896865346,
 835.1307897828692,
 1464.880038387798,
 0.0004788041772373769,
 0.0986984841328713,
 0.0003126583